In [2]:
# 基本
from pathlib import Path
import re, multiprocessing as mp

# 処理
import numpy as np
import nibabel as nib
from scipy.ndimage import gaussian_filter

# 進捗表示
from tqdm.notebook import tqdm


# trainデータにgaussin適用

In [4]:
def make_prob_map(src_path: Path, dst_path: Path, sigma_mm: float = 1.0):
    """
    src_path : cut_ans****.nii / .nii.gz（0/1 マスク）
    dst_path : 出力先（*_prob.nii.gz）
    sigma_mm : ガウス σ [mm]
    """
    img  = nib.load(str(src_path), mmap=True)
    mask = (np.asanyarray(img.dataobj) > 0).astype(np.float32)

    # voxel 単位 σ
    spacing = img.header.get_zooms()[:3]              # (sx, sy, sz) [mm]
    sigma   = tuple(sigma_mm / s for s in spacing)

    prob  = gaussian_filter(mask, sigma=sigma, mode="nearest")
    if prob.max() > 0:
        prob /= prob.max()

    dst_path.parent.mkdir(parents=True, exist_ok=True)
    nib.save(
        nib.Nifti1Image(prob.astype(np.float32), img.affine, img.header.copy()),
        str(dst_path)
    )

INPUT_ROOT  = Path("/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/processed_train")        # ★実パスに置換
OUTPUT_ROOT = Path("/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/proceed_train_prob")   # 出力ルート
SIGMA_MM    = 1.0                                             # ガウス σ
N_WORKERS   = min(8, mp.cpu_count())                          # 並列数 (0=シングル)

regex = re.compile(r"cut_ans\d+\.nii(\.gz)?$")
mask_paths = [
    p for p in INPUT_ROOT.rglob("*") if p.is_file() and regex.match(p.name)
]

print(f"found {len(mask_paths)} mask files")

def _job(args):
    src, dst = args
    make_prob_map(src, dst, SIGMA_MM)
    return dst

jobs = []
for src in mask_paths:
    rel = src.relative_to(INPUT_ROOT)
    out = OUTPUT_ROOT / rel.with_suffix("").with_suffix(".nii.gz")
    out = out.with_name(out.stem + "_prob.nii.gz")
    jobs.append((src, out))

if N_WORKERS > 1:
    with mp.Pool(N_WORKERS) as pool:
        for _ in tqdm(pool.imap_unordered(_job, jobs), total=len(jobs)):
            pass
else:                                # シングルスレッド
    for j in tqdm(jobs):
        _job(j)

print("✓ all probability maps written to", OUTPUT_ROOT)




found 319 mask files


  0%|          | 0/319 [00:00<?, ?it/s]

✓ all probability maps written to /mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/proceed_train_prob
